## **Context**

1. **Project background**：
This project is part of a big new program titled ‘A new Green Revolution: Manipulating the soil microbiome to enhance the sustainability of 21st century agriculture,’. The current focus is to optimise/engineer soil microbiomes to suppress the fungus Gaeumannomyces tritici, which causes the take-all disease in wheat (T. aestivum).

2. **Microbial community coalescence**：
It is the process by which two distinct microbial communities encounter one another and interact to form a new ‘daughter’ community. This assemblage may more closely resemble one ‘parent’ community than the other, and its functional profile may be totally unique. We plan to use a coalescence method to create new community to fight against take-all disease.


## **Obectives**

**Can we predict winners and losers during coalescence?**

This very basic question remains relatively un-explored. However, we did have some insights on community assembly. That is, species with higher CUE value tend to survive. And there is certain connection between species richness and CUE variance.

We will use the similiar methdology as community assembly to simulate the community dynamics during coalescence, mediated by temperature. By doing that, we hope to predict the result of community coalescence through Carbon usage efficiency appearance of species in different temperature. The outcome of this research will give some foundamental information for the coalescence experiments and the dealing with take-all disease project.

**Microbial Consumer Resource Model**

$$
\frac{dC_i}{dt} = \sum_{\alpha=0}^{M} C_i R_{\alpha} u_{i\alpha} (1 - \lambda_{\alpha}) - C_i m_i
$$

$$
\frac{dR_{\alpha}}{dt} = \rho_{\alpha} - R_{\alpha} \omega_{\alpha} - \sum_{i=0}^{N} C_i R_{\alpha} u_{i\alpha} + \sum_{i=0}^{N} \sum_{\beta=0}^{M} C_i R_{\beta} u_{i\beta} l_{\beta\alpha}
$$


| **Parameter** | **Description** | **Key** |
|--------------|---------------|--------|
| $C_i$ | Biomass of the $i$th consumer | - |
| $R_{\alpha}$ | Mass of the $\alpha$th resource | - |
| $N$ | Number of consumer populations | $N$ |
| $M$ | Number of resources | $M$ |
| $u_{i\alpha}$ | Uptake rate of the $\alpha$th resource by the $i$th consumer | $u$ |
| $m_i$ | Loss term for the $i$th consumer | $m$ |
| $\rho_{\alpha}$ | Inflow rate for the $\alpha$th resource | $\rho$ |
| $\omega_{\alpha}$ | Outflow term for the $j$th resource | $\omega$ |
| $l_{\alpha\beta}$ | Proportion of uptake of the $\alpha$th resource leaked to the $\beta$th resource. | $l$ |
| $\lambda_{\alpha}$ | Total proportion of the $\alpha$th resource leaked, same as $\sum_{\beta} l_{\alpha\beta}$ | $\lambda$ |


We adapt this model because it is able to capture the complex interactions of both competition for resources and the exchange of metabolic by-products.

## **Simulation:**

Import the packages

In [ ]:
using Pkg
Pkg.activate("../packages")

create an independent environment to avoid the conflicts

In [ ]:
using Distributions

`Distributions` is a package for probability distributions and associated functions. We need it to use dirichlet distribution for uptake matrix and leakage matrix

In [ ]:
using MiCRM

`MiCRM` is the package for Simulation of Microbial Consumer Resource (MiCRM) systems. 

Now we  *set system size and leakage rate*

In [ ]:
N,M,leakage = 100,50,0.3

N is the **number of species in a community**, M is the **number of resource types**, leakage is the overall leakage rate

Next, *make uptake matrix out of dirichlet distribution*

In [ ]:
u =  MiCRM.Parameters.modular_uptake(M,N; N_modules = 2, s_ratio = 10.0)

The N_modules determines how many the different blocks of resources will be divided into, and s_ratio determines how much will species focus on intra-group resources. *More details about this function see document of functions

*Define other terms*

In [ ]:
#cost term
m = ones(N)

m is a loss term for the the consumers, usually refer to the maintenance; 

In [ ]:
#resource inflow + outflow
ρ,ω = ones(M),ones(M)

ρ is the resource inflow of the system and ω is outflow.

Next, we *make leakage matrix out of dirichlet distribution and make it modular*

In [ ]:
l = MiCRM.Parameters.modular_leakage(M; N_modules = 2, s_ratio = 10.0, λ = leakage)

Similarly to the `MiCRM.Parameters.modular_uptake`, the N_modules determines how many the different blocks of resources will be divided into, and s_ratio determines the structure and probability of leakage.

In [ ]:
#define parameters
param = MiCRM.Parameters.generate_params(N, M;  u = u, m = m, ρ = ρ, ω = ω, l = l, λ = leakage)

We use this function to aggregate the parameters of the system. If only with N, M, and λ, this function will help generate other parameters by default (randomly).

For now, we finished the parameter part and it is time to create an **Ordinary Differential Equation Problem** object which defines the problem for the ODE solver and then solve it with the aptly solve function.
To define the ODEProblem we need to specify the initial state of the system as well as the timespan

In [ ]:
#original state
x0 = ones(N+M)

x0 is an initial state, a one-dimensional array (vector) of length N+M, where all elements are initialized to 1.  
This represents the initial state of the system with N consumer populations and M resource types.

In [ ]:
#time span
tspan = (0.0, 10.0)

The initial time is 0.0 and the final time is 10.0

In [ ]:
#define problem
using DifferentialEquations
prob = ODEProblem(MiCRM.Simulations.dx!, x0, tspan, param)

We import the `DifferentialEquations.jl` package, which provides tools for solving differential equations.  
`MiCRM.Simulations.dx!` function is adopted to define the ODE problem, with initial conditions `x0`, time span `tspan`, and parameters `param` for numerical solving.

In [ ]:
sol = solve(prob, Tsit5())

In this case, we use Tsit5 as a solver, which is widely applicable for non-stiff problems.
If need more information about the solver choice, refer to https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/

At last, we *calculate key properties* to determine whether the system is stable

In [ ]:
# jacobian of system from the solution object
J = MiCRM.Analysis.get_jac(sol)

`MiCRM.Analysis.get_jac(sol)` numerically calculate jacobian of system from the solution object. The Jacobian matrix (J) is a matrix of partial derivatives that represents the local linearization of a system of differential equations. The eigenvalues of J indicate whether equilibria are stable or unstable.

In [ ]:
# generate purbulence matrix
pur = rand(size(J, 1))

Generates a random vector of length equal to the number of rows in J, with each element sampled from a uniform distribution between 0 and 1, which will be used to perturb the system for sensitivity or stability analysis

In [ ]:
# the instantaneous rate of growth of the perturbation pur at time t.
t = 5
MiCRM.Analysis.get_Rins(J, pur, t)

This function computes Instantaneous Resilience (Rins), which measures how quickly a system returns to equilibrium after a disturbance. If there is a positive value, that means the perturbation is growing, therefore the system is locally unstable. If it is negative, then the perturbation is shrinking.

In [ ]:
# Determine the stability a system
MiCRM.Analysis.get_stability(J)

Determine the stability a system given its jaccobian by testing if the real part of the leading eigenvalue is positive. It will return a boolean value.

In [ ]:
#test whether a system is reactive to the perturbation 
MiCRM.Analysis.get_reactivity(J,pur)

Test wether the system is "reactive" to the perturbation u. A reactive is system is one where the initial perturbation is amplified so that the deviation from equilibrium initially increases. It will also return a boolean value.

In [ ]:
#get the rate of return of the system from perturbation
MiCRM.Analysis.get_return_rate(J)

Get the rate of return of the system from perturbation. This value is determined by the leading eigenvalue regardless of the direction of the perturbation or the observation vector we choose.

Visualization

figures

## **Outputs and next step**

Outputs: an assembled stable community, with the information of each species' resource uptake rate and each resources' leakage rate  
Next step: coalescence. We plan to make two stable community respectively and give the traits of these two community to model as input to go through simulation for coalescence.  
The problem is how to link these two community together to make them as a whole.

## **Documents of functions**

**MiCRM.Parameters.modular_uptake**  
`MiCRM.Parameters.modular_uptake(M,N; N_modules = 2, s_ratio = 10.0)`  
It helps generate an uptake matrix using a Dirichlet distribution such that the uptake of all consumers sums to 1.
The number of modules determines how many groups of resources the consumers are specialised over. For example if N_modules = 2 then the resources will be split into two groups with half the consumers specialising on one and half on the other.

The degree of specialisation is determined by the s_ratio value. This controls the relative value of the dirchlet α parameters which determine how the probabiltiy density function is distributed over the different resources. Specialisation is obtained by setting the α values for resources that specialists consume to higher values meaning they have a higher probablity of a larger value. When s_ratio = 1 the proabailtiy is uniform and all resources are equally likely to be consumed. When s_ratio >1 then consumers are more likely to consume resources within thier module.

**MiCRM.Parameters.modular_leakage**  
```MiCRM.Parameters.modular_leakage(M; N_modules = 2, s_ratio = 10.0, λ = leakage)```  
It generates a modular leakage matrix with a directional leakage structure. The matrix is generated using a Dirichlet distribution such that the leakage of all resources sums to λ.

The number of modules determines how many groups of resources they are split into. For example if N_modules = 5 then the resources will be split into five groups with the first group of resources tending to leak to the second, the second to the third and so on.

The degree of to which resources leak in this constrained way (verses randomly across all resources) is determined by the s_ratio value. This controls the relative value of the dirchlet α parameters which determine how the probabiltiy density function is distributed over the different resources. A greater probabiltiy of leakage from one resource to another is obtained by setting the α values for resources to higher values. When s_ratio = 1 the proabailtiy is uniform and all resources are equally likely to be leaked to each other. When s_ratio >1 then resources are more likely to leak to resources thier own or the next module in the sequence.

**MiCRM.Parameters.generate_params**  
```MiCRM.Parameters.generate_params(N, M;  u = u, m = m, ρ = ρ, ω = ω, l = l, λ = leakage)```  
It generates parameter sets for MiCRM simualtions. Requires the system size is defined as well as the functions to generate the actual parameters used in the simulations. Extra arguments can also be passed via the kwargs argument.By default the function generates a random MiCRM system with no structure and uptake and leakage matricies generated by Dirichlet distribitions.

The function returns a NamedTuple with all the parameters as well as a kw entry which itself is a NamedTuple with the additional arguments provided.